# Load the necessary modules.

In [3]:
import jax.numpy as jnp
import rheedium as rh

## Use `autoreload` to reload changed modules, you may disable this for normal use.

In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
structure_file = rh.inout.parse_cif("../tests/test_data/SrTiO3.cif")

In [6]:
structure_file

CrystalStructure(frac_positions=Array([[ 0.5,  0.5,  0.5, 38. ],
       [ 0. ,  0. ,  0.5, 22. ]], dtype=float64), cart_positions=Array([[1.97256507e+00, 1.97256507e+00, 1.97256506e+00, 3.80000000e+01],
       [1.20784775e-16, 1.20784775e-16, 1.97256506e+00, 2.20000000e+01]],      dtype=float64), cell_lengths=Array([3.94513013, 3.94513013, 3.94513013], dtype=float64), cell_angles=Array([90., 90., 90.], dtype=float64))

In [7]:
zone_axis = jnp.array([1, 1, 1])
thickness = jnp.asarray([12, 12, 2])

# Now, load the parameters

In [8]:
voltage_kV = 20.0  # Electron beam voltage in kV
theta_deg = 2.0  # Grazing angle
hmax, kmax, lmax = 4, 4, 2  # Reciprocal lattice indexing bounds
detector_distance = 1000.0  # Detector distance in Å
tolerance = 0.05  # Tolerance for Ewald sphere intersection

# Now, we will process things step by step.

## Electron wavelength (Å)

In [9]:
lam_ang = rh.simul.wavelength_ang(voltage_kV)
lam_ang

Array(0.08585529, dtype=float64, weak_type=True)

## Build real-space lattice vectors

In [10]:
cell_vectors = rh.ucell.build_cell_vectors(
    *structure_file.cell_lengths, *structure_file.cell_angles
)

In [11]:
cell_vectors

Array([[3.94513013e+00, 0.00000000e+00, 0.00000000e+00],
       [2.41569549e-16, 3.94513013e+00, 0.00000000e+00],
       [2.41569549e-16, 2.41569549e-16, 3.94513013e+00]], dtype=float64)

## Generate reciprocal lattice vectors

In [12]:
Gs = rh.ucell.generate_reciprocal_points(
    crystal=structure_file, hmax=hmax, kmax=kmax, lmax=lmax, in_degrees=True
)

## Incident wavevector

In [13]:
k_in = rh.simul.incident_wavevector(lam_ang, theta_deg)

## Allowed reflections by kinematic scattering conditions

In [14]:
allowed_indices, k_out = rh.simul.find_kinematic_reflections(
    k_in=k_in,
    Gs=Gs,
    lam_ang=lam_ang,
)

TypeError: missing a required argument: 'gs'

## Project wavevectors onto the detector plane

In [ ]:
detector_points = simul.project_on_detector(k_out, detector_distance)

## Compute kinematic intensities

In [ ]:
atom_positions = structure_file.cart_positions[:, :3]
G_allowed = Gs[allowed_indices]
intensities = simul.compute_kinematic_intensities(atom_positions, G_allowed)

In [ ]:
pattern = types.RHEEDPattern(
    G_indices=allowed_indices,
    k_out=k_out,
    detector_points=detector_points,
    intensities=intensities,
)

## Plot the RHEED pattern

In [ ]:
plots.plot_rheed(pattern, grid_size=400, interp_type="cubic")

ValueError: zero-size array to reduction operation minimum which has no identity